# 목표 : 머신러닝 돌리기

In [1]:
import pandas as pd

# 직접 분류한 리뷰로 머신러닝 돌리기 => 정확도 95%

In [2]:
sentiment_1 = pd.read_csv("./data/sentiment_1.csv")

In [3]:
sentiment_1 = sentiment_1.drop(["Unnamed: 0"], axis=1)

In [4]:
sentiment_1.head(3)

,review,P/N
0,We’d never had Korean before and I’d been want...,1
1,I really was unsure of how much of the menu wo...,1
2,Absolutely delicious authentic Korean food ser...,1


In [5]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [6]:
import re
import pandas as pd
from time import time

# 전처리 작업을 위해 호출될 함수
def preprocessor(text) :
    # 문자열의 내의 html 태그를 삭제한다.
    # 문자열에서 이모티콘을 찾아낸다.
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)|\^.?\^', str(text))
    # 문장에서 특수문자를 제거하고
    # 문자열을 소문자로 변하고
    # 추출한 이모티콘을 붙혀준다.
    text = re.sub('[\W]+', ' ', str(text).lower() + ' '.join(emoticons).replace('-', ''))
    # print(text)
    return text

In [7]:
sentiment_1["review"] = sentiment_1["review"].apply(preprocessor)

In [8]:
sentiment_1.to_csv("./data/refined_review.csv", index=False)

In [9]:
refined_review = pd.read_csv("./data/refined_review.csv")
refined_review.head()

,review,P/N
0,we d never had korean before and i d been want...,1
1,i really was unsure of how much of the menu wo...,1
2,absolutely delicious authentic korean food ser...,1
3,the banchan or side dishes that they serve are...,1
4,my eleven year old twins beg to get biminbop o...,1


In [10]:
# def step2_preprocessing() :
#     # csv 데이터를 읽어온다.
#     df = pd.read_csv('./data/trip_final.csv')

#     # 전처리 작업
#     stime = time()
#     print('전처리 시작')
#     df["review"] = df['review'].apply(preprocessor)
#     print('전처리 완료')
#     print('소요시간 : %d' % (time() - stime))

#     # 전처리된 데이터를 저장한다.
#     df.to_csv('./data/pre_review.csv', index=False)

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
# 평점 전처리
def star_proprocessing(text) :
    value = int(text)
    if value <= 3.0 :
        return '0'
    else :
        return '1'

In [13]:
def step2_preprocessing():
    # 수집한 데이터를 읽어온다.
    df = pd.read_csv('./data/trip_final.csv')
    # print(df)

    # 전처리 과정
    df['rating'] = df['rating'].apply(star_proprocessing)
    # 학습 데이터와 테스트 데이터로 나눈다.
    text_list = df['review'].tolist()
    star_list = df['rating'].tolist()

    text_train, text_test, star_train, star_test = train_test_split(text_list, star_list, test_size=0.3, random_state=0)
    #print(len(text_train))
    #print(len(text_test))
    #print(len(star_train))
    #print(len(star_test))

    # 저장한다.
    dic_train = {
        'text' : text_train,
        'star' : star_train
    }
    df_tran = pd.DataFrame(dic_train)

    dic_test = {
        'text' : text_test,
        'star' : star_test
    }
    df_test = pd.DataFrame(dic_test)

    df_tran.to_csv('./data/trip_train_data.csv', index=False)
    df_test.to_csv('./data/trip_test_data.csv', index=False)

In [14]:
# step2_preprocessing()

In [15]:
# step3_word_tokenizer.py
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
import nltk

# stopword 단어 사전을 다운로드 받는다.
nltk.download('stopwords')
# stopword 데이터를 가져온다.
stop = stopwords.words('english')
# 단어 줄기를 하기위한 객체
porter = PorterStemmer()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
# 공백으로 단어분리
def tokenizer(text) :
    return text.split()

In [17]:
# 단어줄기
def tokenizer_porter(text) :
    return [porter.stem(word) for word in text.split()]

In [18]:
def tokenizer_stopwordsr(text) :
    # 띄어쓰기를 기준으로 분리한다.
    word_list = text.split()
    #단어 줄기 처리
    word_list2 = \
        [porter.stem(word) for word in word_list]
    #불용어 처리
    result = []
    for w in word_list2: 
        if w not in stop: 
            result.append(w)
    return result

In [19]:
def step3_word_tokenizer() :
    text = 'runners like running and thus they run'

    a1 = tokenizer(text)
    a2 = tokenizer_porter(text)
    print('a1 :', a1)
    print('a2 :', a2)

In [20]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
import pickle
from time import time
import pandas as pd
import os
from sklearn.metrics import accuracy_score

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [22]:
from sklearn.linear_model import LogisticRegression

In [23]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

In [24]:
def step4_learning() :
    # csv 파일에서 데이터를 읽어온다.
    df = pd.read_csv('./data/refined_review.csv')
    # 테스트, 학습데이터로 나눈다.
    X_train = df.loc[:700 - 1, 'review'].values
    y_train = df.loc[:700 - 1, 'P/N'].values

    X_test = df.loc[300:, 'review'].values
    y_test = df.loc[300:, 'P/N'].values

    # 단어장을 만들어주는 객체 생성
    tfidf = TfidfVectorizer(lowercase=False, tokenizer=tokenizer)
    # tfidf = TfidfVectorizer(lowercase=False, tokenizer=tokenizer_stopwordsr)
    # 데이터를 학습하기 위한 객체
    logistic = LogisticRegression(C=10.0, penalty='l2', random_state=0)
    # 파이프 라인 설정
    pipeline = Pipeline([('vect', tfidf), ('clf', logistic)])

    # 학습한다.
    stime = time()
    print('학습 시작')
    pipeline.fit(X_train, y_train)
    print('학습 종료')
    print('총 학습시간 : %d' % (time() - stime))

    # 테스트
    y_pred = pipeline.predict(X_test)
    print("정확도 : %.3f" % accuracy_score(y_test, y_pred))

    # 성능 확인
    y_true = y_test
    y_hat = y_pred
    print("R2 score : ", r2_score(y_true, y_hat))
    print("mean_absolute_error : ", mean_absolute_error(y_true, y_hat))
    print("mean_squared_error : ", mean_squared_error(y_true, y_hat))

    # 학습이 완료된 객체를 저장한다.
    with open('./data/trip.dat', 'wb') as fp :
        pickle.dump(pipeline, fp)

    print('저장완료')

In [26]:
step4_learning()

학습 시작
학습 종료
총 학습시간 : 0
정확도 : 0.951
R2 score :  0.4861405405405407
mean_absolute_error :  0.04864091559370529
mean_squared_error :  0.04864091559370529
저장완료


# 별점으로 긍정부정 머신러닝 학습시키기

In [27]:
sentiment_2 = pd.read_csv("./data/sentiment_2.csv")

In [28]:
sentiment_2 = sentiment_2.drop(["Unnamed: 0"], axis=1)

In [29]:
sentiment_2["review"] = sentiment_2["review"].apply(preprocessor)

In [30]:
sentiment_2.to_csv("./data/refined_review_2.csv", index=False)

In [1]:
refined_review_2 = refined_review_2.dropna(axis=0)

In [ ]:
refined_review_2.to_csv("./data/refined_review_2.csv", index=False)

In [ ]:
refined_review_2 = pd.read_csv("./data/refined_review_2.csv")
refined_review_2.head()

In [ ]:
refined_review_2.isnull().sum()

In [ ]:
refined_review_2

In [ ]:
def step4_learning() :
    # csv 파일에서 데이터를 읽어온다.
    df = pd.read_csv('./data/refined_review_2.csv')
    # 테스트, 학습데이터로 나눈다.
    X_train = df.loc[:35000, 'review'].values
    y_train = df.loc[:35000, 'P/N'].values

    X_test = df.loc[15000:, 'review'].values
    y_test = df.loc[15000:, 'P/N'].values

    # 단어장을 만들어주는 객체 생성
    tfidf = TfidfVectorizer(lowercase=False, tokenizer=tokenizer)
    # tfidf = TfidfVectorizer(lowercase=False, tokenizer=tokenizer_stopwordsr)
    # 데이터를 학습하기 위한 객체
    logistic = LogisticRegression(C=10.0, penalty='l2', random_state=0)
    # 파이프 라인 설정
    pipeline = Pipeline([('vect', tfidf), ('clf', logistic)])

    # 학습한다.
    stime = time()
    print('학습 시작')
    pipeline.fit(X_train, y_train)
    print('학습 종료')
    print('총 학습시간 : %d' % (time() - stime))

    # 테스트
    y_pred = pipeline.predict(X_test)
    print("정확도 : %.3f" % accuracy_score(y_test, y_pred))

    # 성능 확인
    y_true = y_test
    y_hat = y_pred
    print("R2 score : ", r2_score(y_true, y_hat))
    print("mean_absolute_error : ", mean_absolute_error(y_true, y_hat))
    print("mean_squared_error : ", mean_squared_error(y_true, y_hat))

    # 학습이 완료된 객체를 저장한다.
    with open('./data/trip_2.dat', 'wb') as fp :
        pickle.dump(pipeline, fp)

    print('저장완료')

In [34]:
step4_learning()

학습 시작
학습 종료
총 학습시간 : 0
정확도 : 0.951
R2 score :  0.4861405405405407
mean_absolute_error :  0.04864091559370529
mean_squared_error :  0.04864091559370529
저장완료


## 직접 분류한 데이터셋 2000개 가지고 학습 시키기

In [35]:
final_sent = pd.read_csv("./data/final_sent.csv")

In [36]:
final_sent = final_sent.drop(["Unnamed: 0"], axis=1)

In [37]:
final_sent["review"] = final_sent["review"].apply(preprocessor)

In [38]:
final_sent.to_csv("./data/refined_final_sent.csv", index=False)

In [39]:
refined_final_sent = refined_final_sent.dropna(axis=0)

In [40]:
refined_final_sent.to_csv("./data/refined_final_sent.csv", index=False)

In [41]:
refined_final_sent = pd.read_csv("./data/refined_final_sent.csv")
refined_final_sent.head()

,review,PN
0,our first dining experience in a north korean ...,1
1,it was my first meal in north korean restauran...,1
2,food are generally good tried lunch bulkogi gr...,1
3,food is generally good worth mentioning are ki...,1
4,we ve passed by the place a couple times and w...,1


In [42]:
refined_final_sent.isnull().sum()

review    0
PN        0
dtype: int64

In [114]:
def step4_learning() :
    # csv 파일에서 데이터를 읽어온다.
    df = pd.read_csv('./data/refined_final_sent.csv')
    # 테스트, 학습데이터로 나눈다.
    X_train = df.loc[:1400, 'review'].values
    y_train = df.loc[:1400, 'PN'].values

    X_test = df.loc[700:, 'review'].values
    y_test = df.loc[700:, 'PN'].values

    # 단어장을 만들어주는 객체 생성
    tfidf = TfidfVectorizer(lowercase=False, tokenizer=tokenizer)
    # tfidf = TfidfVectorizer(lowercase=False, tokenizer=tokenizer_stopwordsr)
    # 데이터를 학습하기 위한 객체
    logistic = LogisticRegression(C=10.0, penalty='l2', random_state=0)
    
    param_grid = [{"vect__ngram_range" : [(1,1)], "vect__stop_words" : [stopwords, None],
              "vect__tokenizer" : [tokenizer, tokenizer_porter],
              "clf__C" : [1.0, 10.0, 100.0]},
             {"vect__ngram_range" : [(1,1)], "vect__stop_words" : [stopwords, None],
              "vect__tokenizer" : [tokenizer, tokenizer_porter],
              "vect__use_idf" : [False], "vect__norm" : [None],
              "clf__C" : [1.0, 10.0, 100.0]}]
    
    
    # 파이프 라인 설정
    pipeline = Pipeline([('vect', tfidf), ('clf', logistic)])
    gs_Ir_tfidf = GridSearchCV(pipeline, param_grid, scoring="accuracy", cv=5, verbose=1, n_jobs=-1)

    # 학습한다.
    stime = time()
    print('학습 시작')
    gs_Ir_tfidf.fit(X_train, y_train)
    
    print('학습 종료')
    print('총 학습시간 : %d' % (time() - stime))

    # 테스트
    y_pred = gs_Ir_tfidf.predict(X_test)
    print("정확도 : %.3f" % accuracy_score(y_test, y_pred))

    # 성능 확인
    y_true = y_test
    y_hat = y_pred
    print("R2 score : ", r2_score(y_true, y_hat))
    print("mean_absolute_error : ", mean_absolute_error(y_true, y_hat))
    print("mean_squared_error : ", mean_squared_error(y_true, y_hat))
    print("best_params : " , gs_Ir_tfidf.best_params_)

    # 학습이 완료된 객체를 저장한다.
    with open('./data/trip_3.dat', 'wb') as fp :
        pickle.dump(pipeline, fp)

    print('저장완료')

In [97]:
f = open("./data/trip_3.dat", "rb")
model = pickle.load(f)

In [115]:
step4_learning()

학습 시작
Fitting 5 folds for each of 24 candidates, totalling 120 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  59 tasks      | elapsed:    8.6s
[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed:   19.0s finished
C:\Users\admin\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abl', 'absolut', 'accid', 'accommod', 'accumul', 'actual', 'advanc', 'advis', 'age', 'allow', 'alon', 'alreadi', 'alway', 'amplifi', 'ani', 'anoth', 'anymor', 'anyon', 'anyth', 'apart', 'appear', 'arriv', 'aspect', 'assur', 'attent', 'authent', 'avail', 'avarag', 'averag', 'awar', 'be', 'becam', 'becaus', 'befor', 'begin', 'blame', 'block', 'buildingoutsid', 'busi', 'carri', 'celebr', 'cellophan', 'centr', 'central', 'certainli', 'certifi', 'chanc', 'characterist', 'check', 'chock', 'choic', 'choos', 'citi', 'cloth', 'coke', 'come', 'complet', 'complic', 'condit', 'consid', 'correct

학습 종료
총 학습시간 : 20
정확도 : 0.950
R2 score :  0.5415471222396104
mean_absolute_error :  0.05007704160246533
mean_squared_error :  0.05007704160246533
best_params :  {'clf__C': 10.0, 'vect__ngram_range': (1, 1), 'vect__stop_words': ['about', 'also', 'and', 'because', 'box', 'dish', 'dishes', 'etc', 'front', 'cashier', 'husband', 'immediately', 'just', 'menu', 'minutes', 'others', 'our', 'relay', 'section', 'some', 'that', 'the', 'their', 'them', 'then', 'this', 'very', 'walked', 'was', 'were', 'what', 'when', 'women', 'busy', 'gotta', 'been', 'here', 'times', 'and', 'this', 'the', 'are', 'there', 'options', 'had', 'have', 'you', 'find', 'authentic', 'pretty', 'choose', 'was', 'parking ', 'were', 'plenty', 'pot', 'dishes', 'they', 'serve', 'free', 'low', 'day', 'cuz', 'eat', 'box', 'today', 'give', 'chance', 'something', 'else', 'really', 'short', 'wife', 'plate', 'always', 'second', 'time', 'very', 'came', 'out', 'thier', 'bowl', 'table', 'still', 'bottom', 'sunny', 'all', 'add', 'some', 'y

# 최적의 파라미터로 모델 확인하기

In [117]:
# csv 파일에서 데이터를 읽어온다.
df = pd.read_csv('./data/refined_final_sent.csv')
# 테스트, 학습데이터로 나눈다.
X_train = df.loc[:1400, 'review'].values
y_train = df.loc[:1400, 'PN'].values

X_test = df.loc[700:, 'review'].values
y_test = df.loc[700:, 'PN'].values

# 단어장을 만들어주는 객체 생성
tfidf = TfidfVectorizer(lowercase=False, tokenizer=tokenizer)
# tfidf = TfidfVectorizer(lowercase=False, tokenizer=tokenizer_stopwordsr)

Ir_tfidf = Pipeline(["vect"])
# 데이터를 학습하기 위한 객체
logistic = LogisticRegression(C=10.0, penalty='l2', random_state=0)


# 파이프 라인 설정
pipeline = Pipeline([('vect', tfidf), ('clf', logistic)])

# 학습한다.
stime = time()
print('학습 시작')
gs_Ir_tfidf.fit(X_train, y_train)

print('최종 파라미터 계산 종료')
print(gs_Ir_tfidf.best_params_)

clf = gs_Ir_tfidf.best_estimator_
print("테스트 정확도: %.3f" % clf.score(X_test,y_test))

학습 시작
Fitting 5 folds for each of 24 candidates, totalling 120 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  38 tasks      | elapsed:    6.9s
[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed:   20.5s finished
C:\Users\admin\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abl', 'absolut', 'accid', 'accommod', 'accumul', 'actual', 'advanc', 'advis', 'age', 'allow', 'alon', 'alreadi', 'alway', 'amplifi', 'ani', 'anoth', 'anymor', 'anyon', 'anyth', 'apart', 'appear', 'arriv', 'aspect', 'assur', 'attent', 'authent', 'avail', 'avarag', 'averag', 'awar', 'be', 'becam', 'becaus', 'befor', 'begin', 'blame', 'block', 'buildingoutsid', 'busi', 'carri', 'celebr', 'cellophan', 'centr', 'central', 'certainli', 'certifi', 'chanc', 'characterist', 'check', 'chock', 'choic', 'choos', 'citi', 'cloth', 'coke', 'come', 'complet', 'complic', 'condit', 'consid', 'correct

최종 파라미터 계산 종료
{'clf__C': 10.0, 'vect__ngram_range': (1, 1), 'vect__stop_words': ['about', 'also', 'and', 'because', 'box', 'dish', 'dishes', 'etc', 'front', 'cashier', 'husband', 'immediately', 'just', 'menu', 'minutes', 'others', 'our', 'relay', 'section', 'some', 'that', 'the', 'their', 'them', 'then', 'this', 'very', 'walked', 'was', 'were', 'what', 'when', 'women', 'busy', 'gotta', 'been', 'here', 'times', 'and', 'this', 'the', 'are', 'there', 'options', 'had', 'have', 'you', 'find', 'authentic', 'pretty', 'choose', 'was', 'parking ', 'were', 'plenty', 'pot', 'dishes', 'they', 'serve', 'free', 'low', 'day', 'cuz', 'eat', 'box', 'today', 'give', 'chance', 'something', 'else', 'really', 'short', 'wife', 'plate', 'always', 'second', 'time', 'very', 'came', 'out', 'thier', 'bowl', 'table', 'still', 'bottom', 'sunny', 'all', 'add', 'some', 'your', 'kept', 'almost', 'definitly', 'friend', 'recently', 'stopped', 'arrived', 'packed', 'long', 'plus', 'including', 'cellophane', 'slivers', 'w

In [122]:
pickle.dump(gs_Ir_tfidf, open(os.path.join("data", "tok_model.pkl"), "wb"), protocol=4)

In [123]:
curDir = os.getcwd()
clf = pickle.load(open(os.path.join(curDir, "data", "tok_model.pkl"), "rb"))

In [125]:
import pickle
import os
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score

df = pd.read_csv("./data/refined_final_sent.csv")

X_train = df.loc[:1400, 'review'].values
y_train = df.loc[:1400, 'PN'].values

X_test = df.loc[700:, 'review'].values
y_test = df.loc[700:, 'PN'].values

f = open("./data/trip_3.dat", "rb")
model = pickle.load(f)

y_pred = model.predict(X_test)
print("테스트 정확도: % 3f" % accuracy_score(y_test, y_pred))

label = {0:"부정적 의견", 1:"긍정적 의견"}

while True:
    txt = input("영문으로 리뷰를 작성하세요: ")
    if txt == "":
        break
        
    example = [txt]
    pirnt("예측: %s\n확률: %.3f%%" % (label[clf.predict(example)[0]],
                                 np.max(clf.predict_proba(example))*100))

C:\Users\admin\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abl', 'absolut', 'accid', 'accommod', 'accumul', 'actual', 'advanc', 'advis', 'age', 'allow', 'alon', 'alreadi', 'alway', 'amplifi', 'ani', 'anoth', 'anymor', 'anyon', 'anyth', 'apart', 'appear', 'arriv', 'aspect', 'assur', 'attent', 'authent', 'avail', 'avarag', 'averag', 'awar', 'be', 'becam', 'becaus', 'befor', 'begin', 'blame', 'block', 'buildingoutsid', 'busi', 'carri', 'celebr', 'cellophan', 'centr', 'central', 'certainli', 'certifi', 'chanc', 'characterist', 'check', 'chock', 'choic', 'choos', 'citi', 'cloth', 'coke', 'come', 'complet', 'complic', 'condit', 'consid', 'correctli', 'countri', 'coupl', 'courteou', 'cozi', 'crown', 'crystal', 'current', 'custom', 'defin', 'definetli', 'definit', 'definitli', 'defrost', 'delic', 'delv', 'descript', 'dine', 'discov', 'do', 'doe', 'downs

테스트 정확도:  0.949923
영문으로 리뷰를 작성하세요: I am familiar with Korean food. My mother is Japanese, so I grew up around Asian foods. I was excited when I found this little place just a few blocks from where I was working. I was traveling on business on Tampa for the week and my husband came along.\n\nThe place is easy to miss. It looks like a little dive but it is cute inside and has more seating than you think. I liked all of the wood seating. I didn't expect it to look so nice inside. We arrived just before the after-5pm rush-hour so we got drinks and appetizers quickly. Fried squid strips were bland but the sauces they served with changed that. The vegetable dumplings were okay, I would prefer them to be steamed rather than fried.\n\nI knew we'd have some issues with the menu as we are not meat eaters and Korean food incorporates a lot of pork, beef and things like oxtail. Most of the places I have been are willing to substitute tofu or calamari in dishes but they wouldn't. The waitress looke

KeyboardInterrupt: 

# 직접 분류한 데이터셋으로 학습시킨 모델로 긍정 부정 리뷰 나눠보기

In [34]:
# csv 파일에서 데이터를 읽어온다.
df = pd.read_csv("./data/PN_dataset/dataset_1600.csv")
# 테스트, 학습데이터로 나눈다.
X_train = df.loc[:1120, 'review'].values
y_train = df.loc[:1120, 'PN'].values

X_test = df.loc[480:, 'review'].values
y_test = df.loc[480:, 'PN'].values

# 단어장을 만들어주는 객체 생성
tfidf = TfidfVectorizer(lowercase=False, tokenizer=tokenizer)
# tfidf = TfidfVectorizer(lowercase=False, tokenizer=tokenizer_stopwordsr)
# 데이터를 학습하기 위한 객체
logistic = LogisticRegression(C=10.0, penalty='l2', random_state=0)
# 파이프 라인 설정
pipeline = Pipeline([('vect', tfidf), ('clf', logistic)])

# 학습한다.
stime = time()
print('학습 시작')
pipeline.fit(X_train, y_train)
print('학습 종료')
print('총 학습시간 : %d' % (time() - stime))

# 테스트
y_pred = pipeline.predict(X_test)
print("정확도 : %.3f" % accuracy_score(y_test, y_pred))

# 성능 확인
y_true = y_test
y_hat = y_pred
print("R2 score : ", r2_score(y_true, y_hat))
print("mean_absolute_error : ", mean_absolute_error(y_true, y_hat))
print("mean_squared_error : ", mean_squared_error(y_true, y_hat))

# 학습이 완료된 객체를 저장한다.
with open('./data/trip_1600.dat', 'wb') as fp :
    pickle.dump(pipeline, fp)

print('저장완료')

학습 시작
학습 종료
총 학습시간 : 0
정확도 : 0.942
R2 score :  0.7678386350952723
mean_absolute_error :  0.05803571428571429
mean_squared_error :  0.05803571428571429


AttributeError: 'Pipeline' object has no attribute '_best_params'

In [26]:
pickle.dump(pipeline, open(os.path.join("data", "jik_model_1600.pkl"), "wb"), protocol=4)

In [30]:
curDir = os.getcwd()
clf = pickle.load(open(os.path.join(curDir, "data", "jik_model_1600.pkl"), "rb"))

In [28]:
import pickle
import os
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score

df = pd.read_csv("./data/PN_dataset/dataset_1600.csv")

X_train = df.loc[:1120, 'review'].values
y_train = df.loc[:1120, 'PN'].values

X_test = df.loc[480:, 'review'].values
y_test = df.loc[480:, 'PN'].values

curDir = os.getcwd()
clf = pickle.load(open(os.path.join(curDir, "data", "jik_model_1600.pkl"), "rb"))

y_pred = clf.predict(X_test)
print("테스트 정확도: % 3f" % accuracy_score(y_test, y_pred))

label = {0:"부정적 의견", 1:"긍정적 의견"}

while True:
    txt = input("영문으로 리뷰를 작성하세요: ")
    if txt == "":
        break
        
    example = [txt]
    print("예측: %s\n확률: %.3f%%" % (label[clf.predict(example)[0]],
                                 np.max(clf.predict_proba(example))*100))

테스트 정확도:  0.941964
영문으로 리뷰를 작성하세요: Authentic Korean BBQ restaurant--not the AYCE kind! The quality of the meat was good and better than normal AYCE.\r\n\r\nMy husband and I came here for dinner on a Monday evening--we made reservations but definitely not as busy as weekends as we've tried to wait in line in the past but didn't have the patience.\r\n\r\nThe dim lighting made it challenging to cook the food on the grill and know when it would be cooked. I may have undercooked some of it or mixed the tongs and felt a bit sick the next day, so definitely make sure you cook it enough!\r\n\r\nThe side dishes were great. The portions are large enough that one order of any meat is enough for one person, so we got one order of boneless short ribs and one of pork shoulder.  We also ordered a rice noodle soup as we weren't sure if it would be enough food.\r\n\r\nThe total came to about $65-$70 in total without any drinks
예측: 부정적 의견
확률: 57.966%
영문으로 리뷰를 작성하세요: i think it's not that great food
예측: 

## 직접 만든 stopward로 머신러닝 돌려보기

In [48]:
from nltk.stem.porter import PorterStemmer

In [57]:
stopwords = ['about','also', 'and','because','box','dish','dishes','etc','front','cashier', "husband", 'immediately','just','menu', 'minutes','others',
'our','relay','section', 'some', 'that', 'the', 'their','them', 'then', 'this', 'very', 'walked', 'was', 'were','what', 'when', 'women','busy','gotta','been','here', 'times', 'and', 'this','the','are',
'there','options','had','have','you','find','authentic','pretty','choose',
'was','parking ','were','plenty','pot','dishes','they','serve','free','low',
'day','cuz','eat','box','today','give','chance','something','else','really','short',
'wife','plate','always','second','time','very','came','out','thier',
'bowl','table','still','bottom','sunny','all','add','some','your','kept','almost',
'definitly','friend','recently','stopped','arrived','packed','long','plus','including',
'cellophane','slivers','went','night','staying','when','home','wide','now','minutes',
'them','that','other','just','their','range','once','car','drive','miles','wanted',
'stop','along','would','waiting','stay','land','people','brought','half',
'dozen','mixes','each','yeas','locations','found','especially','significantly','plae',
'two','blocks','away','trained','places','quite','can','min','years','started','lived',
'countries','myself','versed','twice','week','owned','own','currently','has','sushi',
'order','correctly','around','couple','sooo','through','sister',
'rest','being','three','ordered','huge','yakimondo','say','portions','large','groups',
'lover','due','neighbor','makes','most','pass','only','looking','word','mouth','building'
'outside','interior','where','welcome','front','opted','guest','another','presentation',
'larger','show','his','work','done','mizo','start','while','did','sample',
'till','open','seats','across','street','which','certainly','proclaim','thought',
'somewhat','amount','navigating','hole','wall','gets','anything','also','ask','how',
'into','probably','put','anyway','yesterday','honestly','whole','its','speak',
'corner','sick','ethnic','everything','coming','kitchen','things','defrosted',
'properly','black','below','average','frozen','let','down','definitely','extremely',
'what','randomly','during','our','managed','past','location','exterior','lot',
'alone','single','piece','gimchi''told','whether','ended','takeaway','aspects',
'dropped','accumulated','mother','law','completely','aged','finding','round','fooled',
'highly','explain','those','who','tourist','itself','themed','decorations','seems',
'delicately','ranges','udon','persons','sets','fix','basement','any','turning',
'reservation','literally','meters','tram','station','plain','view','look',
'fact','stairs','near','helping','after','step','realize','setting','pleasing',
'decor','motives','modern','lines','entered','certify','personal','touch','noticed',
'high','chair','return','weekend','per','noticing','accommodation','book','advance',
'evening','upon','arrival','considered','ourselves','tables','predominantly', 'variations',
'boyfriend','clear','even','joked','asking','prompt','center','longer',
'morning','sight','seeing','courteous','overpaying','gave','mispronounced',
'thoroughly','trio','cokes','walking','dad','took','mums','house','might','comfort',
'mean','isn','either','reminded','higher','because','prime','hours','search','gem',
'required','minute','ride','see','neighborhood','part','flat','city','accept',
'carry','imagine','ready','known','customer','accommodated','requests','daughter',
'owner','patient','help','split','types','non','pesto','beginning','choices',
'end','saved','tongues','every','pay','sized','portion','background','playing',
'person','starter', 'mension','sayd','does','locate','environment',
'true','stated','splendid','she','sorry','expecting','layers','realized',
'redo','both','leaves','slightly','broth','standards','serving','recommending',
'prepared','presented','characteristics','personally','her','meant','immediately',
'much','chat','woman','card','actually','path','means','dress','before','line',
'since','hair','real','speaks','simple','centrally','located','english',
'issue','owners','afternoon','four','warm','biggest','cozy','making','meet',
'enter','definetly','semi','gone','particularly','available',
'walk','downside','crowns','rooms','feels','totally','knew', 'guess',
'advisable','ahead','incredibly','apartment','advisor','heart','warming','door',
'notch','area','somewhere','boxes','absolutely','frequent','eater','reference',
'material','unassuming','arrive','nonetheless','cash','earth','abroad','exquisite',
'reason','conditioning','ventilation','mindblowing','mention','hold','sitting',
'air','ground','customers','days','then','doing','tricky','road','avarage','definately',
'give','soon','everyday','quite','get','cloths','hanger','cute','hidden','part',
'gladly','for','czk','simply','starters','allowed','importantly','mom', 'uber','ride','walk',
'our','group','comes','aunt','owns','grew','hands','school','absolute','having','seen',
'steps','nothing','quiet','partner','tray','etc','searching','lie','finish','centre',
'brought','everyone','older','usually', 'increased','evenings','booking','confirm',
'authenticity','luck','planned','customers','kinds','stuff','centrum','going','smaller',
'shop','overall','mix','already','much','past','month','birthday','celebration','terms',
'delve','below','wise','attentiveness','empty','occassion','revealed','amplified',
'nervous','guests','seated','anyone','hello','mins','happening','third','somehow',
'crowd','youngsters','winter','jacket','live','nearby','substantially','improved',
'jap','identify','blamed','favouring','provides','extreme','only','pictures',
'study','frequently','think','eum','ppong','talking','tap','exactly','remembers',
'visiting','placed','address','speaking','dull','humble','surroundings','cuz',
'light','hospital','suchi','further','relatively','skeptical','chocked','par',
'letdown','tad','bland','dry','why','prohibitively','either','washed','downplay',
'extensive','thing','massive','rush','touched','job','growing','anymore','reservations',
'keep','offering','reading','often','nonsense','number','etnic','extra','assurance',
'decor','thus','maybe','romantic','date','yourself','gentleman','rate','five','perhaps',
'amounts','suspicions','confirmed','sight','crystals','struggle','hungry','taxi',
'man','knows','occasion','able','booked','dined','temperature','host','checked',
'miss','four,','correct','fat','stole','gentle','town','plus','move','stops',
'prepared','played','pop','spice','help','personnel','girlfriend','genuinly','version',
'doubt','occasional','tailored','given','honest','aware','missed','moving','expectations',
'key','easiest','zizkov','staircase','map','eventually','passion','othet','let',
'plates','expecting','arugula','minced','manager','fifteen','complicated',
'ours','stayed','hotel','relation','usual','unbeatable','appears','may','tonight',
'europe','fluent','encountered','truly', 'accident','randomly','living','obsessed',
'goes','cross','moved','became','lazy','whatever','responsive','regard','descriptive',
'listings','maintaining','wooing','evr','tourism','greeted','months','traveling',
'happened','world','recipes','cure','weakness','ower','warmth','afford','fuss',
'forward','else','office','week','esp','item','crowns','via','returned',
'design','discover']

In [64]:
from sklearn.linear_model import LogisticRegression

In [67]:
def step5_learning() :
    # csv 파일에서 데이터를 읽어온다.
    df = pd.read_csv('./data/refined_final_sent.csv')
    # 테스트, 학습데이터로 나눈다.
    X_train = df.loc[:1400, 'review'].values
    y_train = df.loc[:1400, 'PN'].values

    X_test = df.loc[700:, 'review'].values
    y_test = df.loc[700:, 'PN'].values
    
    tfidf = TfidfVectorizer(lowercase=False)
    
    param_grid = [{"vect__ngram_range" : [(1,1)], "vect__stop_words" : [stopwords, None],
                  "vect__tokenizer" : [tokenizer, tokenizer_porter],
                  "clf__C" : [1.0, 10.0, 100.0]},
                 {"vect__ngram_range" : [(1,1)], "vect__stop_words" : [stopwords, None],
                  "vect__tokenizer" : [tokenizer, tokenizer_porter],
                  "vect__use_idf" : [False], "vect__norm" : [None],
                  "clf__C" : [1.0, 10.0, 100.0]}]
    
    Ir_tfidf = Pipeline([("vect", tfidf), ("clf", LogisticRegression(random_state=0))])
    gs_Ir_tfidf = GridSearchCV(Ir_tfidf, param_grid, scoring="accuracy", cv=5, verbose=1, n_jobs=-1)
    
    gs_Ir_tfidf.fit(X_train, y_train)
    
    print("최적 파라미터 계산 종료")
    print(gs_Ir_tfidf.best_params_)
    
    clf = gs_Ir_tfidf.best_estimator_
    print("테스트 정확도 : %.3f" %clf.score(X_test, y_test))

In [68]:
step5_learning()

Fitting 5 folds for each of 24 candidates, totalling 120 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  38 tasks      | elapsed:    6.7s
[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed:   20.0s finished
C:\Users\admin\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abl', 'absolut', 'accid', 'accommod', 'accumul', 'actual', 'advanc', 'advis', 'age', 'allow', 'alon', 'alreadi', 'alway', 'amplifi', 'ani', 'anoth', 'anymor', 'anyon', 'anyth', 'apart', 'appear', 'arriv', 'aspect', 'assur', 'attent', 'authent', 'avail', 'avarag', 'averag', 'awar', 'be', 'becam', 'becaus', 'befor', 'begin', 'blame', 'block', 'buildingoutsid', 'busi', 'carri', 'celebr', 'cellophan', 'centr', 'central', 'certainli', 'certifi', 'chanc', 'characterist', 'check', 'chock', 'choic', 'choos', 'citi', 'cloth', 'coke', 'come', 'complet', 'complic', 'condit', 'consid', 'correct

최적 파라미터 계산 종료
{'clf__C': 10.0, 'vect__ngram_range': (1, 1), 'vect__stop_words': ['about', 'also', 'and', 'because', 'box', 'dish', 'dishes', 'etc', 'front', 'cashier', 'husband', 'immediately', 'just', 'menu', 'minutes', 'others', 'our', 'relay', 'section', 'some', 'that', 'the', 'their', 'them', 'then', 'this', 'very', 'walked', 'was', 'were', 'what', 'when', 'women', 'busy', 'gotta', 'been', 'here', 'times', 'and', 'this', 'the', 'are', 'there', 'options', 'had', 'have', 'you', 'find', 'authentic', 'pretty', 'choose', 'was', 'parking ', 'were', 'plenty', 'pot', 'dishes', 'they', 'serve', 'free', 'low', 'day', 'cuz', 'eat', 'box', 'today', 'give', 'chance', 'something', 'else', 'really', 'short', 'wife', 'plate', 'always', 'second', 'time', 'very', 'came', 'out', 'thier', 'bowl', 'table', 'still', 'bottom', 'sunny', 'all', 'add', 'some', 'your', 'kept', 'almost', 'definitly', 'friend', 'recently', 'stopped', 'arrived', 'packed', 'long', 'plus', 'including', 'cellophane', 'slivers', 'w